# Processamento de Dados

Este notebook encontra-se dividido em três partes. A primeira parte contém todo o processamento de dados efectuado aos dados recolhidos para a construção do dataset daily_covid, e asegunda contém todos os passos aplicados na construção do dataset daily_diabetes.


Para o processamento dos diferentes datasets foram utilizadas as bibliotecas python: numpy e pandas

In [25]:
import numpy as np
import pandas as pd

Devido aos diferentes sinais separadores de atributos dos registos presentes nos datasets utilizados, foi necessário criar funções diferentes para o load dos datasets.

In [26]:
def load_normal_dataset(path):
    return pd.read_csv(path)

In [27]:
def load_datasets(path):
    return pd.read_csv(path, sep=';')

O método to_csv_file, é utilizado para construir o dataset final que posteriormente será utilizado nas previsões.

In [28]:
def to_csv_file(df, file_name):
    df.to_csv(file_name)

## Parte 1 - Daily_covid dataset

Nesta Parte, irei pormenorizar todos as escolhas e passos tomados, para que fosse possível construir um dataset final, capaz de ser robusto o suficiente para eventualmente se obter uma boa previsão para o número de mortes causadas por covid-19 em Portugal. Para a construção deste dataset foram utilizados dados obtidos no repositório [Data Science for Social Good Portugal(DDSGP)](https://github.com/dssg-pt/covid19pt-data), que correspondem a registos diários de mortes por covid e dados atmosféricos obtidos através da plataforma [Visualcrossing](https://www.visualcrossing.com/weather/weather-data-services#/login). Estes dados atmosféricos são as médias diárias para Portugal no mesmo periodo de registos observados no dataset do repositório DSSGP.

### Load do dataset daily_covid_19_portugal

In [29]:
df_raw_covid = load_normal_dataset("daily_datasets/daily_covid_19_portugal.csv")

### Load dos datasets atmosféricos de Portugal

In [30]:
df_raw_temp_2020_01 = load_normal_dataset('daily_datasets/temp_portugal_01012020_31032020.csv')
df_raw_temp_2020_02 = load_normal_dataset('daily_datasets/temp_portugal_01042020_30062020.csv')
df_raw_temp_2020_03 = load_normal_dataset('daily_datasets/temp_portugal_01072020_30092020.csv')
df_raw_temp_2020_04 = load_normal_dataset('daily_datasets/temp_portugal_01102020_31122020.csv')
df_raw_temp_2020_05 = load_normal_dataset('daily_datasets/temp_portugal_01012021_31032021.csv')
df_raw_temp_2020_06 = load_normal_dataset('daily_datasets/temp_portugal_042021.csv')

Após estes loads, foi necessário efetuar tratamento de dados obtidos, nomeadamente tratamento de missing values, alterações dos tipos dos atributos e por fim a concatenação dos diferentes datasets num só dataset.

### Tratamento dos datasets

Em prepare_data_covid, encontra-se todo o tratamento efectuado ao dataset daily_covid_19_portugal. Nesta função foram eliminados atributos cujas colunas apresentavam número de missing válues superiores a 20% da totalidade dos dados. Eliminamos a diferenciação do número de óbitos e infectados por género, para isso somou-se o valor dos atributos correspondentes à mesma idade, resultando em apenas uma coluna correspondente à sua respetiva idade (ex: obitos_20_29m + obitos_20_29f = obitos_20_29, infetados_20_29m + infetados_20_29f = infetados_20_29). Atributos cujo o número de missing values era inferior a 20%, esses missing values foram substituídos pelo valor registado no dia anterior (para missing values referentes a registos não iniciais da série temporal), ou foram substituídos pelo valor 0 (para missing values referentes a registos no periodo inicial da pandemia)

In [31]:
def prepare_data_covid(df_raw_covid):
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    df_aux = df_raw_covid.copy()
    print(df_aux.shape)

    print("Number of missing Values: ")
    print(df_aux.isnull().sum())


    df_aux = df_raw_covid.drop(columns=['data_dados', 'confirmados_estrangeiro', 'lab', 'suspeitos', 'vigilancia',
                                        'n_confirmados', 'cadeias_transmissao', 'transmissao_importada',
                                        'sintomas_tosse', 'sintomas_febre', 'sintomas_cefaleia',
                                        'sintomas_dores_musculares', 'sintomas_dificuldade_respiratoria', 'sintomas_fraqueza_generalizada',
                                        'obitos_estrangeiro', 'recuperados_arsnorte','recuperados_arscentro',
                                        'recuperados_arslvt','recuperados_arsalentejo',
                                        'recuperados_arsalgarve','recuperados_acores', 'recuperados_madeira',
                                        'recuperados_estrangeiro','confirmados_desconhecidos_m','confirmados_desconhecidos_f',
                                        'incidencia_nacional','incidencia_continente','rt_nacional', 'rt_continente',
                                        'obitos_f', 'obitos_m','confirmados_desconhecidos','confirmados_f','confirmados_m',
                                        'internados'], inplace=False)


    #somar colunas casos confirmados com mesma faixa etaria ignorando sexo
    df_aux['confirmados_0_9'] = df_aux.apply(lambda x: x['confirmados_0_9_f'] + x['confirmados_0_9_m'], axis=1)
    df_aux['confirmados_10_19'] = df_aux.apply(lambda x: x['confirmados_10_19_f'] + x['confirmados_10_19_m'], axis=1)
    df_aux['confirmados_20_29'] = df_aux.apply(lambda x: x['confirmados_20_29_f'] + x['confirmados_20_29_m'], axis=1)
    df_aux['confirmados_30_39'] = df_aux.apply(lambda x: x['confirmados_30_39_f'] + x['confirmados_30_39_m'], axis=1)
    df_aux['confirmados_40_49'] = df_aux.apply(lambda x: x['confirmados_40_49_f'] + x['confirmados_40_49_m'], axis=1)
    df_aux['confirmados_50_59'] = df_aux.apply(lambda x: x['confirmados_50_59_f'] + x['confirmados_50_59_m'], axis=1)
    df_aux['confirmados_60_69'] = df_aux.apply(lambda x: x['confirmados_60_69_f'] + x['confirmados_60_69_m'], axis=1)
    df_aux['confirmados_70_79'] = df_aux.apply(lambda x: x['confirmados_70_79_f'] + x['confirmados_70_79_m'], axis=1)
    df_aux['confirmados_80_plus'] = df_aux.apply(lambda x: x['confirmados_80_plus_f'] + x['confirmados_80_plus_m'], axis=1)

    df_aux = df_aux.drop(columns=['confirmados_0_9_f','confirmados_0_9_m',
                            'confirmados_10_19_f','confirmados_10_19_m',
                            'confirmados_20_29_f','confirmados_20_29_m',
                            'confirmados_30_39_f','confirmados_30_39_m',
                            'confirmados_40_49_f','confirmados_40_49_m',
                            'confirmados_50_59_f','confirmados_50_59_m',
                            'confirmados_60_69_f','confirmados_60_69_m',
                            'confirmados_70_79_f','confirmados_70_79_m',
                            'confirmados_80_plus_f','confirmados_80_plus_m'], inplace=False)


    #somar colunas obitos confirmados com mesma faixa etaria ignorando sexo
    df_aux['obitos_0_9'] = df_aux.apply(lambda x: x['obitos_0_9_f'] + x['obitos_0_9_m'], axis=1)
    df_aux['obitos_10_19'] = df_aux.apply(lambda x: x['obitos_10_19_f'] + x['obitos_10_19_m'], axis=1)
    df_aux['obitos_20_29'] = df_aux.apply(lambda x: x['obitos_20_29_f'] + x['obitos_20_29_m'], axis=1)
    df_aux['obitos_30_39'] = df_aux.apply(lambda x: x['obitos_30_39_f'] + x['obitos_30_39_m'], axis=1)
    df_aux['obitos_40_49'] = df_aux.apply(lambda x: x['obitos_40_49_f'] + x['obitos_40_49_m'], axis=1)
    df_aux['obitos_50_59'] = df_aux.apply(lambda x: x['obitos_50_59_f'] + x['obitos_50_59_m'], axis=1)
    df_aux['obitos_60_69'] = df_aux.apply(lambda x: x['obitos_60_69_f'] + x['obitos_60_69_m'], axis=1)
    df_aux['obitos_70_79'] = df_aux.apply(lambda x: x['obitos_70_79_f'] + x['obitos_70_79_m'], axis=1)
    df_aux['obitos_80_plus'] = df_aux.apply(lambda x: x['obitos_80_plus_f'] + x['obitos_80_plus_m'], axis=1)

    df_aux = df_aux.drop(columns=['obitos_0_9_f','obitos_0_9_m',
                            'obitos_10_19_f','obitos_10_19_m',
                            'obitos_20_29_f','obitos_20_29_m',
                            'obitos_30_39_f','obitos_30_39_m',
                            'obitos_40_49_f','obitos_40_49_m',
                            'obitos_50_59_f','obitos_50_59_m',
                            'obitos_60_69_f','obitos_60_69_m',
                            'obitos_70_79_f','obitos_70_79_m',
                            'obitos_80_plus_f','obitos_80_plus_m'], inplace=False)

    #drop das ultimas 2 rows do dataset
    df_aux = df_aux.drop(df_aux.tail(2).index)

    #fill row com missing values aplicando ffill method, n vejo problema pois aind a não haviam casos e faltando um dia mantem o valor do dia anterio
    #obitos_80_plus
    df_aux = df_aux.fillna(method='ffill', limit=1)
    #obitos_arsalentejo
    df_aux = df_aux.fillna(method='ffill', limit=19)
    #como os nan estão ao inicio da pandemia, é natural n terem valor, logo vou substituir esses NaN por 0
    df_aux = df_aux.fillna(0)

    #print(df_aux.isnull().sum())
    #print(df_aux)
    #print("Number of missing Values: ")
    #print(df_aux.isnull().sum())
    # counting the duplicates
    dups = df_aux.pivot_table(index=['data','confirmados'], aggfunc='size')
    # displaying the duplicate Series
    print(dups)
    #Não tem duplicados


    df_aux.columns = ['Date', 'confirmados', 'confirmados_arsnorte', 'confirmados_arscentro',
     'confirmados_arslvt', 'confirmados_arsalentejo',
     'confirmados_arsalgarve', 'confirmados_acores', 'confirmados_madeira',
     'confirmados_novos', 'recuperados', 'obitos', 'internados_uci',
     'obitos_arsnorte', 'obitos_arscentro', 'obitos_arslvt',
     'obitos_arsalentejo', 'obitos_arsalgarve', 'obitos_acores',
     'obitos_madeira', 'ativos', 'internados_enfermaria', 'confirmados_0_9',
     'confirmados_10_19', 'confirmados_20_29', 'confirmados_30_39',
     'confirmados_40_49', 'confirmados_50_59', 'confirmados_60_69',
     'confirmados_70_79', 'confirmados_80_plus', 'obitos_0_9',
     'obitos_10_19', 'obitos_20_29', 'obitos_30_39', 'obitos_40_49',
     'obitos_50_59', 'obitos_60_69', 'obitos_70_79', 'obitos_80_plus']


    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    #print(df_aux.columns)
    #print(df_aux)

    return df_aux

In [32]:
df_data_covid = prepare_data_covid(df_raw_covid)

(424, 93)
Number of missing Values: 
data                                   0
data_dados                             0
confirmados                            0
confirmados_arsnorte                   0
confirmados_arscentro                  0
confirmados_arslvt                     0
confirmados_arsalentejo                0
confirmados_arsalgarve                 0
confirmados_acores                     0
confirmados_madeira                    0
confirmados_estrangeiro              408
confirmados_novos                      0
recuperados                            0
obitos                                 0
internados                             8
internados_uci                        17
lab                                  260
suspeitos                            251
vigilancia                             7
n_confirmados                        269
cadeias_transmissao                  409
transmissao_importada                257
confirmados_0_9_f                      8
confirmados_0_9_m   

Em append_tem_dataframes, procedeu-se à junção de todos os datasets referentes aos dados atmosféricos de Portugal, para ser possível um tratamento mais fácil desses datasets.

In [33]:
def append_temp_dataframes(df_raw_temp_2020_01, df_raw_temp_2020_02, df_raw_temp_2020_03, df_raw_temp_2020_04,
                           df_raw_temp_2020_05, df_raw_temp_2020_06):
    df_raw = df_raw_temp_2020_01.append(df_raw_temp_2020_02, ignore_index = True)
    df_raw = df_raw.append(df_raw_temp_2020_03, ignore_index = True)
    df_raw = df_raw.append(df_raw_temp_2020_04, ignore_index = True)
    df_raw = df_raw.append(df_raw_temp_2020_05, ignore_index = True)
    df_raw = df_raw.append(df_raw_temp_2020_06, ignore_index = True)

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    #print(df_raw)

    return df_raw

In [34]:
df_raw_appended_temp = append_temp_dataframes(df_raw_temp_2020_01,df_raw_temp_2020_02,df_raw_temp_2020_03,df_raw_temp_2020_04,df_raw_temp_2020_05,df_raw_temp_2020_06)


Na função prepare_data_temp, encontra-se todo o processamento aplicado aos dados atmosféricos. Aqui, são eliminadas as coluna com missing values superiores a 80% da totalidade dos registos observados, e é executadado one-hot-encoding ao atributo Conditions através de aplicação da função apply().

In [35]:
def condition_ohe_rain(condition):
    res =0
    if(condition == 'Rain' or condition == 'Rain, Partially cloudy'):
        res=1

    return res




In [36]:
def condition_ohe_clear(condition):
    res =0
    if(condition == 'Clear'):
        res=1

    return res




In [37]:
def condition_ohe_cloudy(condition):
    res =0
    if(condition == 'Rain, Partially cloudy' or condition == 'Partially cloudy'):
        res=1

    return res

In [38]:
def prepare_data_temp(df_raw):

    df_aux = df_raw.copy()
    #Ver se tem missing values
    mv = df_aux.isnull().sum()
    print(mv)


    #drop de colunas com muitos missing values e a coluna nome do país, colunas referentes á neve(têm apenas valores ='s a 0)
    df_aux = df_aux.drop(columns=['Name','Wind Chill','Heat Index', 'Wind Gust', 'Snow', 'Snow Depth'])

    #rename columns
    df_aux.columns = ['Date', 'Max_Temp','Min_Temp','Temperature','Precipitation','Wind_Speed','Wind_Direction','Visibility','Cloud_Cover','Relative_Humidity','Conditions']

    #one hot encoding weather conditions
    df_aux['Rain'] = np.nan
    df_aux['Clear'] = np.nan
    df_aux['Partially_cloudy'] = np.nan
    
    df_aux['Rain'] = df_aux.apply(lambda x: condition_ohe_rain(x['Conditions']), axis=1)
    df_aux['Clear'] = df_aux.apply(lambda x: condition_ohe_clear(x['Conditions']), axis=1)
    df_aux['Partially_cloudy'] = df_aux.apply(lambda x: condition_ohe_cloudy(x['Conditions']), axis=1)

    df_aux = df_aux.drop(columns='Conditions')
    #colocar data igual ao dataset daily_covid_19_portugal
    df_aux["Date"] = pd.to_datetime(df_aux["Date"]).dt.strftime('%d-%m-%Y')

    return df_aux


In [39]:
df_data_temp = prepare_data_temp(df_raw_appended_temp)

Name                     0
Date time                0
Maximum Temperature      0
Minimum Temperature      0
Temperature              0
Wind Chill             400
Heat Index             399
Precipitation            0
Snow                     0
Snow Depth               0
Wind Speed               0
Wind Direction           0
Wind Gust              343
Visibility               0
Cloud Cover              0
Relative Humidity        0
Conditions               0
dtype: int64


Após o processamentodos diferentes datasets, procedeu-se à concatenação dos mesmos através da operação merge(df_data_covid,df_data_temp,on='Date',how='left'), uma vez os registos atmosféricos são de um periodo temporal superior aos registos temporais da pandemia.

In [40]:
def unifie_covid_datasets(df_data_covid, df_data_temp):

    df_covid = pd.merge(df_data_covid,df_data_temp, on='Date', how='left')

    print(df_data_covid.shape)
    print(df_data_temp.shape)

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    return df_covid


In [41]:
df_covid = unifie_covid_datasets(df_data_covid, df_data_temp)

(422, 40)
(486, 13)


Por fim é necessário criar o dataset final, que será utilizaos nas previsões sobre as mortes por covid em Portugal.

In [42]:
df_covid["Date"] = pd.to_datetime(df_covid["Date"])
df_covid = df_covid.set_index('Date')
df_covid.head()

,confirmados,confirmados_arsnorte,confirmados_arscentro,confirmados_arslvt,confirmados_arsalentejo,confirmados_arsalgarve,confirmados_acores,confirmados_madeira,confirmados_novos,recuperados,obitos,internados_uci,obitos_arsnorte,obitos_arscentro,obitos_arslvt,obitos_arsalentejo,obitos_arsalgarve,obitos_acores,obitos_madeira,ativos,internados_enfermaria,confirmados_0_9,confirmados_10_19,confirmados_20_29,confirmados_30_39,confirmados_40_49,confirmados_50_59,confirmados_60_69,confirmados_70_79,confirmados_80_plus,obitos_0_9,obitos_10_19,obitos_20_29,obitos_30_39,obitos_40_49,obitos_50_59,obitos_60_69,obitos_70_79,obitos_80_plus,Max_Temp,Min_Temp,Temperature,Precipitation,Wind_Speed,Wind_Direction,Visibility,Cloud_Cover,Relative_Humidity,Rain,Clear,Partially_cloudy
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-02-26,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.4,10.0,12.9,0.10,22.3,325.25,12.3,7.5,68.15,1,0,0
2020-02-27,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.4,10.4,14.5,0.00,16.9,321.58,11.8,3.9,72.08,0,1,0
2020-02-28,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.1,10.3,13.3,0.00,17.7,123.33,10.1,3.8,79.20,0,1,0
2020-02-29,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,13.1,15.1,3.53,24.7,253.13,9.6,6.8,82.29,1,0,0
2020-01-03,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.1,14.0,15.3,0.68,38.6,235.63,9.5,4.3,92.37,1,0,0


In [ ]:
to_csv_file(df_covid, "daily_covid.csv")

## Parte 2 - Daily_diabetes dataset

Aqui, são pormenorizas todas as escolhas e passos tomados, para que fosse possível construir um dataset final, capaz de ser robusto o suficiente para eventualmente se obter uma boa previsão para o número de mortes causadas por diabetes em Inglaterra. Para a construção deste dataset foram utilizados dados sobre a temperatura em Inglaterra através do [European Climate Assessment & Dataset](https://www.ecad.eu/dailydata/customquery.php), o registo diário do nível medio de Ozono através [UK Air - Department for environment Food & Rural Affairs](https://uk-air.defra.gov.uk/interactive-map), e o registo de mortes por diabetes no ano de 2014 através de [England Office for National Statistics](https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/adhocs/005259dailydeathoccurrencesbyallcausesanddiabetesmellitusicd10e10toe14englandandwales2012to2014)

### Load do dataset Diabetes_2014_England

In [21]:
df_raw_diabetes = load_datasets("daily_datasets/Diabetes_2014_England.csv")

### Load dos datasets das temperaturas em Inglaterra

Para podermos utilizar temperaturas médias registadas em Inglaterra, foi necessário recorrer a várias estações meteorológicas espalhas poir Inglaterra, as estações escolhidas foram as estações de Blackpool, Durham, Nottingham e Bognor Regis.

In [22]:
df_raw_temp_bognor_regis = load_normal_dataset("daily_datasets/mean_temp_bognor_regis.csv")
df_raw_temp_blackpool = load_normal_dataset("daily_datasets/mean_temp_blackpool.csv")
df_raw_temp_durham = load_normal_dataset("daily_datasets/mean_temp_durham.csv")
df_raw_temp_nottingham = load_normal_dataset("daily_datasets/mean_temp_nottingham.csv")

### Load dos datasets dos n+iveis de Ozono em Ingalterra

Para podermos utilizar os níveis médios de ozono registadas em Inglaterra, foi necessário recorrer a várias estações meteorológicas espalhas poir Inglaterra, as estações escolhidas foram as estações de Hull (Freetown), Liverpool (Wirral Tranmere), London (Bloomsbury) e Norwich (Lakenfields).

In [23]:
df_raw_ozone_hull = load_normal_dataset("daily_datasets/hull_freetown_ozone.csv")
df_raw_ozone_norwich = load_normal_dataset("daily_datasets/norwich_lakenfields_ozone.csv")
df_raw_ozone_wirral = load_normal_dataset("daily_datasets/wirral_tranmere_ozone.csv")
df_raw_ozone_london = load_normal_dataset("daily_datasets/london_ozone.csv")

Após estes loads, foi necessário efetuar tratamento de dados obtidos, nomeadamente tratamento de missing values, alterações dos tipos dos atributos, cálculo de média de medidas de ozono e temperatura e por fim a concatenação dos diferentes datasets num só dataset.

### Tratamento dos datasets